# Limpieza de los datos de la Base de datos `bookmaker.db`

Antes de empezar a desarrollar la limpieza de los datos, hemos seleccionado como Kernel el entorno que hemos creado previamente

In [1]:
%pip install numpy
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

C:\Users\mglez\AppData\Local\Temp\ipykernel_8148\2253275120.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Ahora vamos a desglosar la base de datos en 6 tabblas, separadas unas de otras

In [3]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('bookmaker.db')

In [4]:
conn.commit()

clientes = pd.read_sql_query("SELECT * FROM clientes", conn)
empresa = pd.read_sql_query("SELECT * FROM empresa", conn)
equipos = pd.read_sql_query("SELECT * FROM equipos", conn)
partidos = pd.read_sql_query("SELECT * FROM partidos", conn)
apuestas = pd.read_sql_query("SELECT * FROM apuestas", conn)
cuotas = pd.read_sql_query("SELECT * FROM cuotas", conn)

tablas = [clientes, empresa, equipos, partidos, apuestas, cuotas]
